Create Domain.json

In [3]:
!pip install ucimlrepo

/home/tommy/miniconda3/envs/DataPrivacy/lib/python3.12/pty.py:95: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [ ]:
import json, numpy as np, pandas as pd
from ucimlrepo import fetch_ucirepo

adult = fetch_ucirepo(id=2)
df = adult.data.features.assign(income=adult.data.targets.squeeze())
df = df.replace("?", np.nan).fillna("0")

domain = {}

skip = {"fnlwgt", "capital-gain", "capital-loss", "age", "hours-per-week"}
for col in df.columns:
    if col in skip:
        continue
    domain[col] = sorted(df[col].astype(str).unique().tolist())

domain["age_bin"] = sorted(pd.cut(
    df["age"].astype(int),
    bins=[17,25,35,50,65, df["age"].astype(int).max()+1],
    labels=False, include_lowest=True
).unique().tolist())

domain["hours_bin"] = sorted(pd.cut(
    df["hours-per-week"].astype(int),
    bins=[0,20,40,60,80, df["hours-per-week"].astype(int).max()+1],
    labels=False, include_lowest=True
).unique().tolist())

domain["income"] = ["<=50K", ">50K"]

with open("domain.json","w",encoding="utf-8") as f:
    json.dump(domain, f, indent=2, ensure_ascii=False)

for k, v in domain.items():
    print(f"{k} ({len(v)}): {v[:5]} …")


Create discrete data

In [ ]:
import json
import pandas as pd
from scipy.constants import ounce

def discretize_adults(input_csv, domain_json, output_csv):
    # Load domain information
    with open(domain_json, encoding="utf-8") as f:
        domain = json.load(f)
    
    # Load the dataset
    df = pd.read_csv(input_csv)
    
    # Replace missing values with "0"
    df = df.replace("?", pd.NA).fillna("0")
    
    # Create age bins
    if "age" in df.columns and "age_bin" in domain:
        df["age_bin"] = pd.cut(
            df["age"].astype(int),
            bins=[17,25,35,50,65, df["age"].astype(int).max()+1],
            labels=False, include_lowest=True
        )
    
    # Create hours bins
    if "hours-per-week" in df.columns and "hours_bin" in domain:
        df["hours_bin"] = pd.cut(
            df["hours-per-week"].astype(int),
            bins=[0,20,40,60,80, df["hours-per-week"].astype(int).max()+1],
            labels=False, include_lowest=True
        )
    
    # Copy dataframe for coding
    df_code = df.copy()
    
    # Map each column according to domain
    for col, vals in domain.items():
        if col in df_code.columns:
            mapping = {str(v): i for i, v in enumerate(vals)}
            df_code[col] = df_code[col].astype(str).map(mapping).fillna(0).astype(int)
    
    # Keep only columns in domain.json
    df_code = df_code[list(domain.keys())]
    
    # Remove specific columns if they exist
    columns_to_drop = ["fnlwgt", "capital-gain", "capital-loss", "age", "hours-per-week"]
    df_code = df_code.drop(columns=[c for c in columns_to_drop if c in df_code.columns])
    
    # Save the discretized dataset
    df_code.to_csv(output_csv, index=False)
    print(f"Discretized dataset saved to {output_csv}")
    print(f"Final shape: {df_code.shape}")
    
    # Show sample
    print("\nSample of discretized data:")
    print(df_code.head())

input_csv = "adult.csv"  
domain_json = "domain.json"
output_csv = "adult_discretized.csv"
discretize_adults(input_csv, domain_json, output_csv)



Discretized dataset saved to adult_discretized.csv
Final shape: (48842, 12)

Sample of discretized data:
   workclass  education  education-num  marital-status  occupation  \
0          7          9              4               4           1   
1          6          9              4               2           4   
2          4         11             15               0           6   
3          4          1             13               2           6   
4          4          9              4               2          10   

   relationship  race  sex  native-country  income  age_bin  hours_bin  
0             1     4    1              39       0        2          1  
1             0     4    1              39       0        2          0  
2             1     4    1              39       0        2          1  
3             0     2    1              39       0        3          1  
4             5     2    0               5       0        1          1  


In [1]:
from mechanism import Mechanism
import mbi
from mbi import Domain, Factor, Dataset
import matrix
import argparse
import numpy as np
from scipy import sparse, optimize
from privacy.research.hyperparameters_2022.rdp_accountant import compute_rdp, get_privacy_spent
from functools import reduce
import os

def transform_data(data, supports):
    df = data.df.copy()
    newdom = {}
    for col in data.domain:
        support = supports[col]
        size = support.sum()
        newdom[col] = int(size)
        if size < support.size:
            newdom[col] += 1
        mapping = {}
        idx = 0
        for i in range(support.size):
            mapping[i] = size
            if support[i]:
                mapping[i] = idx
                idx += 1
        assert idx == size
        df[col] = df[col].map(mapping)
    newdom = Domain.fromdict(newdom)
    return Dataset(df, newdom)

def reverse_data(data, supports):
    df = data.df.copy()
    newdom = {}
    for col in data.domain:
        support = supports[col]
        mx = support.sum()
        newdom[col] = int(support.size)
        idx, extra = np.where(support)[0], np.where(~support)[0]
        mask = df[col] == mx
        if extra.size == 0:
            pass
        else:
            df.loc[mask, col] = np.random.choice(extra, mask.sum())
        df.loc[~mask, col] = idx[df.loc[~mask, col]]
    newdom = Domain.fromdict(newdom)
    return Dataset(df, newdom)

def moments_calibration(round1, round2, eps, delta):
    # round1: L2 sensitivity of round1 queries
    # round2: L2 sensitivity of round2 queries
    # works as long as eps >= 0.01; if larger, increase orders
    orders = range(2, 4096)

    def obj(sigma):
        rdp1 = compute_rdp(1.0, sigma/round1, 1, orders)
        rdp2 = compute_rdp(1.0, sigma/round2, 1, orders)
        rdp = rdp1 + rdp2
        privacy = get_privacy_spent(orders, rdp, target_delta=delta)
        return privacy[0] - eps + 1e-8
    low = 1.0
    high = 1.0
    while obj(low) < 0:
        low /= 2.0
    while obj(high) > 0:
        high *= 2.0
    sigma = optimize.bisect(obj, low, high)
    assert obj(sigma) - 1e-8 <= 0, 'not differentially private' # true eps <= requested eps
    return sigma


In [2]:
import json
import pandas as pd
import numpy as np
import sys
from mbi import Domain, Dataset
from match3 import Match3
import mbi.estimation as estimation 
import matrix
from scipy import sparse
from privacy.research.hyperparameters_2022 import rdp_accountant
import random
from inference import FactoredInference

class AdultsSetup(Match3):
    def __init__(self, dataset_path, specs_path, iters=1000):
        super().__init__(dataset_path, specs_path, iters=iters)
        
        with open('domain.json', 'r') as f:
            domain_dict = json.load(f)
            
        self.epsilon = 1.0 
        self.delta = 2.2820544e-12
    
    def setup(self):
        """為 Adult 數據集定制的 setup 方法"""
        self.round1 = list(self.domain.attrs)
        
        self.round2 = [
            ('workclass', 'income'),
            ('education', 'income'),
            ('marital-status', 'income'),
            ('occupation', 'income'),
            ('age_bin', 'income'),
            ('hours_bin', 'income'),
            ('sex', 'income'),
            ('education', 'occupation'),
            ('age_bin', 'occupation'),
            ('sex', 'occupation'),
            ('marital-status', 'relationship'),
            ('native-country', 'income'),
            ('race', 'income'),
            
            ('age_bin', 'education', 'income'),
            ('age_bin', 'occupation', 'income'),
            ('sex', 'occupation', 'income'),
            ('education', 'occupation', 'income'),
            ('workclass', 'occupation', 'income'),
            ('marital-status', 'relationship', 'income')
        ]
        
        domain_attrs = set(self.domain.attrs)
        self.round2 = [rel for rel in self.round2 if all(attr in domain_attrs for attr in rel)]
        
        print("Setup completed!")
        print(f"Round1 queries: {len(self.round1)} one-way marginals")
        print(f"Round2 queries: {len(self.round2)} multi-way marginals")
        
        print("\nSample two-way relationships:")
        for i, rel in enumerate(self.round2[:5]):
            print(f"  {i+1}. {rel}")
    
    def measure(self):
        """為 Adult 數據集定制的測量方法"""
        print("開始測量過程...")
        data = self.load_data()
        
        # 計算噪聲參數 (基於隱私預算)
        sigma = moments_calibration(1.0, 1.0, self.epsilon, self.delta)
        print('噪聲水平 (sigma):', sigma)

        # 計算邊際的權重，使 L2 敏感度為 1
        weights = np.ones(len(self.round1))
        weights /= np.linalg.norm(weights)  # 現在 L2 範數 = 1

        supports = {}
  
        self.measurements = []
        # 處理 Round1 查詢（一階邊際）
        for col, wgt in zip(self.round1, weights):
            # 噪聲添加步驟
            proj = (col,)
            hist = data.project(proj).datavector()
            noise = sigma * np.random.randn(hist.size)
            y = wgt * hist + noise
          
            # 後處理步驟
            # 確定支持集（頻率高於噪聲閾值的值）
            sup = y >= 3 * sigma
            supports[col] = sup
            print(f"{col}: 域大小={hist.size}, 支持集大小={sup.sum()}")

            # 重構查詢結果
            if sup.sum() == y.size:
                y2 = y
                I2 = matrix.Identity(y.size)
            else:
                # 合併低頻值
                y2 = np.append(y[sup], y[~sup].sum())
                I2 = np.ones(y2.size)
                I2[-1] = 1.0 / np.sqrt(y.size - y2.size + 1.0)
                y2[-1] /= np.sqrt(y.size - y2.size + 1.0)
                I2 = sparse.diags(I2)

            # 保存測量結果
            self.measurements.append((I2, y2/wgt, 1.0/wgt, proj))

        # 保存支持集信息
        self.supports = supports 
        
        # 基於支持集轉換數據以減少稀疏性 
        data = transform_data(data, supports)
        self.domain = data.domain

        # 過濾掉太大的查詢 (避免計算爆炸)
        self.round2 = [cl for cl in self.round2 if self.domain.size(cl) < 1e6]
        weights = np.ones(len(self.round2))
        weights /= np.linalg.norm(weights)  # 現在 L2 範數 = 1
   
        # 處理 Round2 查詢（二階和三階邊際）
        for proj, wgt in zip(self.round2, weights):
            # 噪聲添加步驟
            hist = data.project(proj).datavector()
            Q = matrix.Identity(hist.size)
            noise = sigma * np.random.randn(Q.shape[0])
            y = wgt * Q.dot(hist) + noise
            self.measurements.append((Q, y/wgt, 1.0/wgt, proj))
            
        print(f"測量完成！生成了 {len(self.measurements)} 個測量結果")
     
    def postprocess(self):
        """Implements McKenna's approach for generating synthetic data"""
        import mbi.callbacks
        from functools import reduce
        from callbacks import Logger
        from mbi import Factor
        import os
        
        # Initialize the inference engine
        domain = self.domain
        engine = FactoredInference(domain,
                                structural_zeros={},
                                iters=500,
                                log=True,
                                warm_start=True,
                                elim_order=None)  # You can set a custom elimination order if needed
        
        self.engine = engine
        cb = Logger(engine)
        
        # Warmup for faster convergence (initialize with one-way marginals)
        engine._setup(self.measurements, None)
        oneway = {}
        for i in range(len(self.round1)):
            p = self.round1[i]
            y = self.measurements[i][1]
            y = np.maximum(y, 1)  # Ensure no negative values
            y /= y.sum()  # Normalize
            oneway[p] = Factor(self.domain.project(p), y)
        
        # Initialize model with product of one-way marginals
        marginals = {}
        for cl in engine.model.cliques:
            marginals[cl] = reduce(lambda x,y: x*y, [oneway[p] for p in cl])
        
        # Maximum likelihood estimation for the initial potentials
        theta = engine.model.mle(marginals)
        engine.potentials = theta
        engine.marginals = engine.model.belief_propagation(theta)
        
        # Create checkpoint file path
        checkpt = self.save[:-4] + '-checkpt.csv'
        
        # Run inference in batches
        for i in range(self.iters // 500):
            engine.infer(self.measurements, engine='MD', callback=cb)
            
            # Save checkpoint every 4 batches (2000 iterations)
            if i % 4 == 3:
                self.synthetic = engine.model.synthetic_data()
                self.synthetic = reverse_data(self.synthetic, self.supports)
                self.transform_domain()
                self.synthetic.to_csv(checkpt, index=False)
        
        # Clean up checkpoint file if it exists
        if os.path.exists(checkpt):
            os.remove(checkpt)
        
        # Generate final synthetic data
        self.synthetic = engine.model.synthetic_data()
        self.synthetic = reverse_data(self.synthetic, self.supports)
        
        # Transform back to original domain
        self.transform_domain()
        
        return self.synthetic
    
if __name__ == "__main__":
    dataset_path = 'adult_discretized.csv'
    specs_path   = 'adult_specs.json'
    adults_setup = AdultsSetup(dataset_path, specs_path)
    adults_setup.setup()

    adults_setup.epsilon = 10
    adults_setup.delta   = 1e-5
    adults_setup.save    = 'adult_synthetic.csv'            

    adults_setup.measure()
    synth_data = adults_setup.postprocess()
    synth_data.to_csv(adults_setup.save, index=False)
    

Codebook inconsistent for education-num
Setup completed!
Round1 queries: 12 one-way marginals
Round2 queries: 19 multi-way marginals

Sample two-way relationships:
  1. ('workclass', 'income')
  2. ('education', 'income')
  3. ('marital-status', 'income')
  4. ('occupation', 'income')
  5. ('age_bin', 'income')
開始測量過程...
噪聲水平 (sigma): 0.7596780918920558
workclass: 域大小=9, 支持集大小=0
education: 域大小=16, 支持集大小=0
education-num: 域大小=17, 支持集大小=16
marital-status: 域大小=7, 支持集大小=0
occupation: 域大小=15, 支持集大小=0
relationship: 域大小=6, 支持集大小=0
race: 域大小=5, 支持集大小=0
sex: 域大小=2, 支持集大小=0
native-country: 域大小=42, 支持集大小=0
income: 域大小=2, 支持集大小=0
age_bin: 域大小=5, 支持集大小=5
hours_bin: 域大小=5, 支持集大小=5
測量完成！生成了 31 個測量結果
錯誤: unhashable type: 'numpy.ndarray'
嘗試備用方案...


ValueError: axis 1 is out of bounds for array of dimension 1

Test synthetic dataset

In [ ]:
#Load the synthetic data
synth_data = pd.read_csv("adult_synthetic.csv")

#Check if have null
print(synth_data.isnull().sum())

df_x = synth_data.drop(columns=["income"])
df_y = synth_data["income"]
print(df_x.shape, df_y.shape)

workclass         0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
native-country    0
income            0
age_bin           0
hours_bin         0
dtype: int64
(48842, 11) (48842,)


In [ ]:
from sklearn.model_selection import train_test_split
from operator import le
from sklearn import preprocessing

seed = 42

x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.2, random_state=seed)

cate = x_train.select_dtypes(include="object").columns
print(cate)


for col in cate:
    all_values = pd.concat([x_train[col], x_test[col]]).unique()
    le = preprocessing.LabelEncoder()
    le.fit(all_values)

    x_train[col] = le.transform(x_train[col])
    x_test[col] = le.transform(x_test[col])


y_train_encoded = (y_train == '>50K').astype(int)
y_test_encoded = (y_test == '>50K').astype(int)


Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country'],
      dtype='object')


In [ ]:
print(x_train.head())
print(type(x_train))
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

scalar = preprocessing.StandardScaler()
x_train = scalar.fit_transform(x_train)
x_test = scalar.transform(x_test)


print(x_train.shape, x_test.shape)
print(type(x_train), type(x_test))
print(x_train[:5])

       workclass  education  education-num  marital-status  occupation  \
37193          4          3              7               6          14   
31093          4          5              1               6          12   
33814          5          6              7               4          11   
14500          2          7              7               2           1   
23399          3          6              1               2           7   

       relationship  race  sex  native-country  age_bin  hours_bin  
37193             5     4    0              23        2          4  
31093             5     3    0              17        0          4  
33814             5     1    1              31        2          4  
14500             4     0    1              13        2          4  
23399             4     0    0              14        0          4  
<class 'pandas.core.frame.DataFrame'>
(39073, 11) (9769, 11) (39073,) (9769,)
(39073, 11) (9769, 11)
<class 'numpy.ndarray'> <class 'numpy.nd

In [ ]:
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score

model = RandomForestClassifier(n_estimators=100, random_state=42)

with open('RandomForeast.pkl', 'wb') as file:
    pickle.dump(model, file)
    random_forest = RandomForestClassifier(
        random_state=seed,
        n_estimators=100,
        max_depth=15,
        min_samples_split=5,
        min_samples_leaf=2,
        )
    random_forest.fit(x_train, y_train)
    y_train_pred = random_forest.predict(x_train)
    train_error_rate = 1 - accuracy_score(y_train, y_train_pred)
    print(f"Train Error Rate: {train_error_rate}")
    y_pred = random_forest.predict(x_test)

print(f"Report: {classification_report(y_test, y_pred)}")
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"Precision: {precision_score(y_test, y_pred, average='weighted')}")
print(f"Recall: {recall_score(y_test, y_pred, average='weighted')}")
print(f"F1 Score: {f1_score(y_test, y_pred, average='weighted')}")

Train Error Rate: 0.02508125815780715
Report:               precision    recall  f1-score   support

       <=50K       0.50      0.45      0.48      4918
        >50K       0.49      0.54      0.52      4851

    accuracy                           0.50      9769
   macro avg       0.50      0.50      0.50      9769
weighted avg       0.50      0.50      0.50      9769

Accuracy: 0.49728733749616133
Precision: 0.49758596707742253
Recall: 0.49728733749616133
F1 Score: 0.4963648007217378
